# Analysis of presence of variants in phylogeny

## Setup

In [3]:
import pandas as pd
from ete4 import Tree

In [4]:
mydb = '/FastData/czirion/Crypto_Diversity_Pipeline/Crypto_Desjardins/results_2024-10-22/02.Dataset/database.db'
tree_path = '/FastData/czirion/Crypto_Diversity_Pipeline/analyses/variants/data/CryptoDiversity_Desjardins_Tree.tre'

## Get variant presence

In [5]:
vars_presence_raw = pd.read_csv('/FastData/czirion/Crypto_Diversity_Pipeline/analyses/variants/data/vars_presence.csv')

Separate unique and non unique variants

In [6]:
vars_num_strains = vars_presence_raw.groupby('var_id')['strain'].nunique().reset_index(name='num_strains')
unique_vars = vars_num_strains[vars_num_strains['num_strains'] == 1]
non_unique_vars = vars_num_strains[vars_num_strains['num_strains'] > 1]
len(unique_vars), len(non_unique_vars), len(vars_num_strains)

(356496, 844318, 1200814)

In [7]:
unique_vars_presence = vars_presence_raw[vars_presence_raw['var_id'].isin(unique_vars['var_id'])].copy()
unique_vars_presence['status'] = "unique"
vars_presence = vars_presence_raw[vars_presence_raw['var_id'].isin(non_unique_vars['var_id'])].copy()

Group the variants present in the same strains to simplify the analysis

In [8]:
grouped_vars = vars_presence.groupby('var_id')['strain'].apply(tuple).reset_index()
grouped_vars = grouped_vars.groupby('strain')['var_id'].apply(list).reset_index()
grouped_vars['grouped_var_id'] = ['group_' + str(i) for i in range(len(grouped_vars))]
grouped_vars

,strain,var_id,grouped_var_id
0,"(125.91, AD1-83a)",[var_VNI_201289],group_0
1,"(125.91, Bt161)",[var_VNI_190507],group_1
2,"(125.91, Bt2)",[var_VNI_26831],group_2
3,"(125.91, D17-1)",[var_VNI_28905],group_3
4,"(125.91, Gbc39-2, Gbc39-1)",[var_VNI_28981],group_4
...,...,...,...
255122,"(Ze90-1, PMHc1049.THER1.STOR, Muc387-2, Bt46, ...","[var_VNBII_304219, var_VNBII_304220, var_VNBII...",group_255122
255123,"(Ze90-1, PMHc1049.THER1.STOR, Muc387-2, Bt46, ...","[var_VNBII_304233, var_VNBII_304237, var_VNBII...",group_255123
255124,"(Ze90-1, PMHc1049.THER1.STOR, NRHc5008.ENR)","[var_VNBII_318521, var_VNBII_318523, var_VNBII...",group_255124
255125,"(Ze90-1, PMHc1049.THER1.STOR, PMHc1009.ENR, Bt...",[var_VNBII_336105],group_255125


Recreate the original dataframe with the grouped variant IDs

In [9]:
var_id_to_grouped_var_id = {}
for _, row in grouped_vars.iterrows():
    for var_id in row['var_id']:
        var_id_to_grouped_var_id[var_id] = row['grouped_var_id']
group_var_presence = vars_presence.copy()
group_var_presence['grouped_var_id'] = group_var_presence['var_id'].map(var_id_to_grouped_var_id)

group_var_presence

,var_id,strain,grouped_var_id
1,var_VNBII_288,PMHc1052.ENR.STOR,group_231054
2,var_VNBII_466,PMHc1052.ENR.STOR,group_244616
3,var_VNBII_502,PMHc1052.ENR.STOR,group_227880
4,var_VNBII_504,PMHc1052.ENR.STOR,group_240741
5,var_VNBII_680,PMHc1052.ENR.STOR,group_240731
...,...,...,...
19001933,var_VNBI_218816,Bt96,group_153543
19001934,var_VNBI_218817,Bt96,group_153543
19001935,var_VNBI_218819,Bt96,group_153678
19001936,var_VNBI_218821,Bt96,group_153656


Make the absence/presence matrix of each group of variants in each strain


In [10]:
groups_presence = group_var_presence.copy()
groups_presence.drop(columns=['var_id'], inplace=True)
groups_presence.drop_duplicates(inplace=True)
groups_presence['presence'] = "1"
groups_presence

,strain,grouped_var_id,presence
1,PMHc1052.ENR.STOR,group_231054,1
2,PMHc1052.ENR.STOR,group_244616,1
3,PMHc1052.ENR.STOR,group_227880,1
4,PMHc1052.ENR.STOR,group_240741,1
5,PMHc1052.ENR.STOR,group_240731,1
...,...,...,...
19001927,Bt96,group_153542,1
19001928,Bt96,group_153543,1
19001929,Bt96,group_153541,1
19001936,Bt96,group_153656,1


In [11]:
vars_matrix = groups_presence.pivot(index='grouped_var_id', columns='strain', values='presence').fillna("0")
vars_matrix['H99'] = "0"
vars_matrix

strain,125.91,8-1,A1-35-8,A1-84-14,A2-102-5,A3-1-1,A3-38-20,A4-1-12,A4-34-6,A5-35-17,...,Tu406-1,Tu416-1,Tu422-1,Ug2459,Ug2462,Ug2463,WM148,WM626,Ze90-1,H99
grouped_var_id,,,,,,,,,,,,,,,,,,,,,
group_0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
group_1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
group_10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
group_100,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
group_1000,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
group_99995,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
group_99996,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
group_99997,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0


## Identify the phylogenetic status of each group of variants

In [12]:
t = Tree(open(tree_path))

Add the presence/absence matrix to the phylogeny

In [ ]:
for var in vars_matrix.index:
    for leaf in t:
        leaf.add_prop(**{var: vars_matrix.loc[var, leaf.name]})        

In [29]:
import itertools

# Generate all possible three-letter combinations
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
three_letter_combinations = [''.join(comb) for comb in itertools.product(letters, repeat=2)]

# Select the first 387 combinations
three_letter_list = three_letter_combinations[:388]

print(three_letter_list)

['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DJ', 'DK', 'DL', 'DM', 'DN', 'DO', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DU', 'DV', 'DW', 'DX', 'DY', 'DZ', 'EA', 'EB', 'EC', 'ED', 'EE', 'EF', 'EG', 'EH', 'EI', 'EJ', 'EK', 'EL', 'EM', 'EN', 'EO', 'EP', 'EQ', 'ER', 'ES', 'ET', 'EU', 'EV', 'EW', 'EX', 'EY', 'EZ', 'FA', 'FB', 'FC', 'FD', 'FE', 'FF', 'FG', 'FH', 'FI', 'FJ', 'FK', 'FL', 'FM', 'FN', 'FO', 'FP', 'FQ', 'FR', 'FS', 'FT', 'FU', 'FV', 'FW', 'FX', 'FY', 'FZ', 'GA', 'GB', 'GC', 'GD', 'GE', 'GF', 'GG', 'GH', 'GI', 'GJ', 'GK

In [30]:
for i, leaf in enumerate(t):
    leaf.add_prop("color", three_letter_list[i])

In [32]:
print(t.to_str(props=["name","color"], show_internal=False))


   ╭╴Bt17,AA
 ╭─┤
 │ ╰╴Gbc51-2,AB
 │
 │   ╭╴Bt209,AC
 │   │
 │   │   ╭╴Bt87,AD
 │   │ ╭─┤
 │   │ │ ╰╴Gbc16-1,AE
 │   │ │
 │   │ │     ╭╴Bt126,AF
 │   │ │   ╭─┤
 │   │ │   │ │ ╭╴Bt90,AG
 │   │ │   │ ╰─┤
 │   │ │   │   │ ╭╴Bt57,AH
 │   │ │   │   ╰─┤
 │   │ │   │     ╰╴Bt62,AI
 │   │ │   │
 │   │ │   │     ╭╴Bt47,AJ
 │   │ │   │   ╭─┤
 │   │ │   │   │ ╰╴Bt8,AK
 │   │ │   │ ╭─┤
 │   │ │   │ │ │ ╭╴Bt118,AL
 │   │ │   │ │ ╰─┤
 │   │ │   │ │   │ ╭╴Bt106,AM
 │   │ │   │ │   ╰─┤
 │   │ │   │ │     ╰╴Bt15,AN
 │   │ │   │ │
 │   │ │   │ │   ╭╴Bt123,AO
 │   │ │ ╭─┤ │   │
 │   │ │ │ │ │   │     ╭╴Bt155,AP
 │   │ │ │ │ │   │   ╭─┤
 │   │ │ │ │ │ ╭─┤   │ ╰╴Bt72,AQ
 │   │ │ │ │ │ │ │ ╭─┤
 │   │ │ │ │ │ │ │ │ │ ╭╴PMHc1031A.ENR.INI.LP,AR
 │   │ │ │ │ │ │ │ │ ╰─┤
 │   │ │ │ │ │ │ │ │   │ ╭╴Bt23,AS
 │   │ │ │ │ │ │ ╰─┤   ╰─┤
 │   │ │ │ │ │ │   │     ╰╴Bt77,AT
 │   │ │ │ │ │ │   │
 │   │ │ │ │ │ │   │ ╭╴Bt83,AU
 │   │ │ │ │ │ │   ╰─┤
 │   │ │ │ │ │ │     │ ╭╴Bt117,AV
 │   │ │ │ │ │ │     ╰─┤
 │   │ │ │ │ │

Check the phylogenetic status of each group of variants

In [46]:
# status_dict = {}
# for var in vars_matrix.index:
#     print(var)
#     status = t.check_monophyly(values="1", prop=var)
#     status_dict[var] = status[1]
t.check_monophyly(values=["AJ", "AK"], prop="color", unrooted=True)


AttributeError: 'ete4.core.tree.Tree' object has no attribute 'color'

In [29]:
status_df = pd.DataFrame.from_dict(status_dict, orient='index', columns=['status']) 
status_df   

,status
group_0,polyphyletic
group_1,polyphyletic
group_10,polyphyletic
group_100,polyphyletic
group_1000,polyphyletic
...,...
group_99995,polyphyletic
group_99996,polyphyletic
group_99997,polyphyletic
group_99998,polyphyletic


Add the status to the original presence dataframe

In [84]:
variant_status = group_var_presence.merge(status_df, left_on='grouped_var_id', right_index=True)
variant_status.drop(columns=['grouped_var_id'], inplace=True)
variant_status = pd.concat([variant_status, unique_vars_presence])
variant_status.to_csv('/FastData/czirion/Crypto_Desjardins/fungal_pop/data/variant_status.csv')
variant_status

,var_id,strain,status
1,var_VNBII_288,PMHc1052.ENR.STOR,polyphyletic
1947,var_VNBII_14675,PMHc1052.ENR.STOR,polyphyletic
10765,var_VNBII_62267,PMHc1052.ENR.STOR,polyphyletic
10770,var_VNBII_62294,PMHc1052.ENR.STOR,polyphyletic
10819,var_VNBII_62553,PMHc1052.ENR.STOR,polyphyletic
...,...,...,...
18998833,var_VNBI_195687,Bt96,unique
18998844,var_VNBI_195913,Bt96,unique
18998848,var_VNBI_195946,Bt96,unique
18998849,var_VNBI_195948,Bt96,unique


Number of variants with each status

In [65]:
unique_vars = variant_status[variant_status['status'] == 'unique']['var_id'].nunique()
mono_vars = variant_status[variant_status['status'] == 'monophyletic']['var_id'].nunique()
para_vars = variant_status[variant_status['status'] == 'paraphyletic']['var_id'].nunique()
poly_vars = variant_status[variant_status['status'] == 'polyphyletic']['var_id'].nunique()
print(f"Unique variants: {unique_vars}")
print(f"Monophyletic variants: {mono_vars}")
print(f"Paraphyletic variants: {para_vars}")
print(f"Polyphyletic variants: {poly_vars}")
print(f"Total non-unique variants: {mono_vars + para_vars + poly_vars}")
print(f"Total variants: {mono_vars + para_vars + poly_vars + unique_vars}")


Unique variants: 356496
Monophyletic variants: 175688
Paraphyletic variants: 14305
Polyphyletic variants: 654325
Total non-unique variants: 844318
Total variants: 1200814


## Testing

In [ ]:
t =  Tree("((((((a, e), i), o),h), u), ((f, g), j));")
print(t)
print(t.check_monophyly(values=["a", "e", "i", "o", "u"], target_attr="name", ))
print(t.check_monophyly(values=["a", "e", "i", "o"], target_attr="name"))
print(t.check_monophyly(values=["i", "o"], target_attr="name"))



                  /-a
               /-|
            /-|   \-e
           |  |
         /-|   \-i
        |  |
      /-|   \-o
     |  |
   /-|   \-h
  |  |
  |   \-u
--|
  |      /-f
  |   /-|
   \-|   \-g
     |
      \-j
(False, 'polyphyletic', {Tree node 'h' (0x73aeac2e47f)})
(True, 'monophyletic', set())
(False, 'paraphyletic', {Tree node 'e' (-0x7ffff8c4fb3fe5af), Tree node 'a' (0x73b04c01a4e)})


In [ ]:
t =  Tree("((((((4, e), i), o),h), u), ((3, 4), (i, june)));")
# we annotate the tree using external data
colors = {"a":"red", "e":"green", "i":"yellow",
          "o":"black", "u":"purple", "4":"green",
          "3":"yellow", "1":"white", "5":"red",
          "june":"yellow"}
for leaf in t:
    leaf.add_features(color=colors.get(leaf.name, "none"))
print(t.get_ascii(attributes=["name", "color"], show_internal=False))

print("Green-yellow clusters:")
# And obtain clusters exclusively green and yellow
for node in t.get_monophyletic(values=["green", "yellow"], target_attr="color"):
   print(node.get_ascii(attributes=["color", "name"], show_internal=False))

In [ ]:
import pandas as pd
import numpy as np

# Create a mock dataframe
mock_data = {
    'var_id': [f'var_{i}' for i in range(1, 11)],
    'a': np.random.choice([0, 1], size=10),
    'b': np.random.choice([0, 1], size=10),
    'c': np.random.choice([0, 1], size=10),
    'd': np.random.choice([0, 1], size=10),
    'e': np.random.choice([0, 1], size=10),
    'f': np.random.choice([0, 1], size=10),
    'g': np.random.choice([0, 1], size=10),
    'h': np.random.choice([0, 1], size=10),
    'i': np.random.choice([0, 1], size=10),
    'j': np.random.choice([0, 1], size=10)
}

mock_df = pd.DataFrame(mock_data)
# Convert integers to strings in the mock dataframe
mock_df = mock_df.astype(str)
mock_df.set_index('var_id', inplace=True)
print(mock_df)


In [ ]:
t =  Tree("((((((a, b), c), d),e), f), ((g, h), (i, j)));")


for var in mock_df.index:
    for leaf in t:
        leaf.add_features(**{var: mock_df.loc[var, leaf.name]})
    status = t.check_monophyly(values="1", target_attr=var)
    status_dict = {var: status[1]}

    print(status_dict)        
        
